In [ ]:
#need to get gauss1d working collumn 2
#on the big space at the bottom row

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
pi = math.pi

In [ ]:
def matlab_style_gauss2D(shape=(3,3),sigma=0.5):
    """
    2D gaussian mask - should give the same result as MATLAB's
    fspecial('gaussian',[shape],[sigma])
    """
    m,n = [(ss-1.)/2. for ss in shape]
    y,x = np.ogrid[-m:m+1,-n:n+1]
    h = np.exp( -(x*x + y*y) / (2.*sigma*sigma) )
    h[ h < np.finfo(h.dtype).eps*h.max() ] = 0
    sumh = h.sum()
    if sumh != 0:
        h /= sumh
    return h

def matlab_style_log2D(shape=(3,3),sigma=0.5):
    """
    2D gaussian mask - should give the same result as MATLAB's
    fspecial('gaussian',[shape],[sigma])
    """
    m,n = [(ss-1.)/2. for ss in shape]
    y,x = np.ogrid[-m:m+1,-n:n+1]
    h = np.exp( -(x*x + y*y) / (2.*sigma*sigma) )
    h[ h < np.finfo(h.dtype).eps*h.max() ] = 0
    sumh = h.sum()
    if sumh != 0:
        h /= sumh
    h1 = h*(x*x + y*y - 2.*sigma*sigma)/(sigma*sigma**2)
    h = h1 - sum(h1)/(shape[0]*shape[1])  
    return h

In [ ]:
#function f=makefilter(scale,phasex,phasey,pts,sup)
#  gx=gauss1d(3*scale,0,pts(1,:),phasex);
#  gy=gauss1d(scale,0,pts(2,:),phasey);
#  f=normalise(reshape(gx.*gy,sup,sup));
#return
def makefilter(scale,phasex,phasey,pts,sup):
    gx=gauss1d(3*scale,0,pts[0,:],phasex);
    gy=gauss1d(scale,0,pts[1,:],phasey);
    f=normalise(np.reshape(gx*gy,(sup-1,sup-1)));
    return f

In [ ]:
#function g=gauss1d(sigma,mean,x,ord)
#% Function to compute gaussian derivatives of order 0 <= ord < 3
#% evaluated at x.
#  x=x-mean;num=x.*x;
#  variance=sigma^2;
#  denom=2*variance; 
#  g=exp(-num/denom)/sqrt(pi*denom);
#  switch ord,
#    case 1, g=-g.*(x/variance);
#    case 2, g=g.*((num-variance)/(variance^2));
#  end;
#return
def gauss1d(sigma,mean,x,ord):
  x=x-mean
  num=x*x
  variance=sigma**2
  denom=2*variance
  g=np.exp(-num/denom)/np.sqrt(pi*denom)
  if (ord ==1):
    g=-g*(x/variance)
  elif (ord == 2):
    g=g*((num-variance)/(variance^2));
  return g

In [ ]:
#function f=normalise(f), f=f-mean(f(:)); f=f/sum(abs(f(:))); return
def normalise(f):
    f = f-np.mean(f)
    f = f/np.sum(np.abs(f))
    return f

In [ ]:
#  SUP=49;                 % Support of the largest filter (must be odd)
#  SCALEX=[1,2,4];         % Sigma_{x} for the oriented filters
#  NORIENT=6;              % Number of orientations
#  NROTINV=2;
#  NBAR=length(SCALEX)*NORIENT;
#  NEDGE=length(SCALEX)*NORIENT;
#  NF=NBAR+NEDGE+NROTINV;
#  F=zeros(SUP,SUP,NF);
#  hsup=(SUP-1)/2;
sup = 12
scalex = np.array([1,2])
norient =4
nrotinv = 2
nbar=len(scalex)*norient
nedge=len(scalex)*norient
nf=nbar+nedge+nrotinv
f = np.zeros([sup-1,sup-1,nf])
hsup = (sup-1)/2;

In [ ]:
#  [x,y]=meshgrid([-hsup:hsup],[hsup:-1:-hsup]);
#  orgpts=[x(:) y(:)]';
xin = np.arange(-hsup, hsup, 1)
yin = np.arange(hsup,-hsup,-1)
x,y = np.meshgrid(xin,yin)
orgpts=np.array([x, y])
orgpts = np.reshape(orgpts,(2,(sup-1)**2))  ######needs work...



In [ ]:
#  count=1;
#  for scale=1:length(SCALEX),
#    for orient=0:NORIENT-1,
#      angle=pi*orient/NORIENT;  % Not 2pi as filters have symmetry
#      c=cos(angle);s=sin(angle);
#      rotpts=[c -s;s c]*orgpts;
#      F(:,:,count)=makefilter(SCALEX(scale),0,1,rotpts,SUP);
#      F(:,:,count+NEDGE)=makefilter(SCALEX(scale),0,2,rotpts,SUP);
#      count=count+1;
#    end;
#  end; 

count=0
for scale  in range( len(scalex)):
    for orient in range(norient):
        angle=math.pi*orient/norient;  #Not 2pi as filters have symmetry
        c=math.cos(angle)
        s=math.sin(angle)      
        rotpts=np.matmul(np.array([[c, -s],[s, c]]),orgpts)
        f[:,:,count]=makefilter(scalex[scale],0,1,rotpts,sup)
        f[:,:,count+nedge]=makefilter(scalex[scale],0,2,rotpts,sup)
        count = count+1

       

In [ ]:
#  F(:,:,NBAR+NEDGE+1)=normalise(fspecial('gaussian',SUP,10));
#  F(:,:,NBAR+NEDGE+2)=normalise(fspecial('log',SUP,10));

f[:,:,nbar+nedge]=normalise(matlab_style_gauss2D((sup-1,sup-1),10))
f[:,:,nbar+nedge+1]=normalise(matlab_style_log2D((sup-1,sup-1),10))
        


In [2]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import math
import cv2
import numpy as np
%matplotlib inline

fig, axs = plt.subplots(6,8, figsize=(15, 6))
fig.subplots_adjust(hspace = .2, wspace=.001)
axs = axs.ravel()
for i in range(38):
    axs[i].axis('off')
    axs[i].imshow(f[:,:,i])
    axs[i].set_title(i)

In [ ]:
def plot_figures(figures, nrows = 1, ncols=1, labels=None):
    fig, axs = plt.subplots(ncols=ncols, nrows=nrows, figsize=(12, 14))
    axs = axs.ravel()
    for index, title in zip(range(len(figures)), figures):
        axs[index].imshow(figures[title], plt.gray())
        if(labels != None):
           axs[index].set_title(labels[index])
        else:
            axs[index].set_title(title)
            
        axs[index].set_axis_off()
        
    plt.tight_layout()

figures = {}
labels = {}
my_signs = []
index = 0
for i in range(18):
    #img = cv2.cvtColor(cv2.imread(my_image), cv2.COLOR_BGR2RGB)
    #my_signs.append(f[:,:,i])
    figures[index] = f[:,:,i]
    #labels[index] = name_values[my_labels[index]][1].decode('ascii')
    index += 1

plot_figures(figures, 3, 7)


# Load pickled data
import pickle
import numpy as np
# TODO: Fill this in based on where you saved the training and testing data



### Data exploration visualization code goes here.
### Feel free to use as many code cells as needed.

import random
import numpy as np
import matplotlib.pyplot as plt
import cv2
%matplotlib inline

index = random.randint(0, len(X_train))

image = X_train[index].squeeze()
#img=cv2.cvtColor(image,cv2.COLOR_BGRA2BGR)[:, :, np.newaxis]
print(image.shape)


#image = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)
#plt.figure(figsize=(15,5))
plt.imshow(image)
#plt.imshow(img)



index = random.randint(0, len(X_train))
image = X_train[index].squeeze()
img = image
kernel = np.ones((5,5),np.float32)/25

dst = cv2.filter2D(img,-1,f[:,:,1])

plt.subplot(131),plt.imshow(img),plt.title('Original')
plt.xticks([]), plt.yticks([])
plt.subplot(132),plt.imshow(dst),plt.title('Averaging')
plt.xticks([]), plt.yticks([])
plt.subplot(133),plt.imshow(f[:,:,1]),plt.title('filter')
plt.xticks([]), plt.yticks([])
plt.show()

In [3]:
fh = open("tiny-imagenet-200/wnids.txt")
count = 0
yval = []
for line in fh:
    nl = ''
    for e in line:
        if e == '\n':
            break
        else:
            nl = nl+e
    yval.append(nl)
    #print(line)
    count+=1
fh.close()
print(count)


os.chdir('tiny-imagenet-200/train')
#list of folders
t2 = os.listdir()
#yval of list


cwd = os.getcwd()

#gets all x values... need to make y[200] oneshot array
#change t2[:10] for all pics and change x size
class_size = 20
size = 500*class_size*2


x = np.zeros([size,64,64,3], dtype='uint8')
y = np.zeros([size,class_size])
c = 0
c2 = 0
for d in t2[:20]:
    #print(d)
    
    for f2 in os.listdir(d+'/images'):
        x[c,:,:,:]=cv2.imread(cwd+'\\'+d+'\\images\\'+f2)
        y[c,c2] = 1
        x[c+1,:,:,:]=cv2.flip(cv2.imread(cwd+'\\'+d+'\\images\\'+f2))
        y[c+1,c2] = 1
        c = c+2
        
        
    c2+=1   
        
        #print(f2)

200


TypeError: Required argument 'flipCode' (pos 2) not found

In [4]:
os.chdir('..')
os.chdir('..')

fh = open("tiny-imagenet-200/val/val_annotations.txt")
count = 0
yval = []
yspot = []
for line in fh:
    nl = ''
    nl2 = ''
    for e in line:
        if (e == '\t'):
            count +=1
        elif (count ==0):
            nl2 = nl2 +e
        elif (count == 1): 
            nl = nl+e
        elif count > 1: 
            break
    yval.append(nl)
    yspot.append(nl2)
    #print(line)
    count = 0
fh.close()


#os.chdir('..')
#os.chdir('..')

os.chdir('tiny-imagenet-200/val/images')
#list of folders
#t3 = os.listdir()
#yval of list


cwd = os.getcwd()

#gets all x values... need to make y[200] oneshot array
#change t2[:10] for all pics and change x size
class_size = 20
sizev = class_size * 50


xv = np.zeros([sizev,64,64,3], dtype='uint8')
yv = np.zeros([sizev,class_size])
c = 0
c2 = 0
for d in yspot:
    #print(d)
    
    #for f2 in os.listdir(d+'/images'):
    if  t2.index(yval[c2]) <=19:
        #xv.append(cwd+'\\'+d)
        xv[c,:,:,:] =cv2.imread(cwd+'\\'+d)
        yv[c, t2.index(yval[c2])] = 1
        #y[c,c2] = 1
        c = c+1
        
    c2+=1   
  

In [5]:
yv[100]

array([ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [6]:
from sklearn.utils import shuffle

n_classes = class_size
print(y.shape)
print(x.shape)
x, y = shuffle(x, y)

(20000, 20)
(20000, 64, 64, 3)


In [7]:
from sklearn.utils import shuffle



#X_test = np.sum(X_test/3, axis=3, keepdims=True)
def preprocess(img):
    #img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_out= img.astype('float32')
    #img_out=(img_out-img_out.mean())/img_out.std()
    img_out=img_out/255
    

    return img_out

x= preprocess(x)
xv= preprocess(xv)
#X_valid, y_valid = preprocess(X_valid, y_valid)
#X_test, y_test = preprocess(X_test, y_test)

z = 0
figures = np.empty([size,64,64,54], dtype='float32')
for image in x:    
    
    
    
    for i in range(18):
        figures[z,:,:,i*3:i*3+3] =  cv2.filter2D(image,-1,f[:,:,i])
        
        #figur =  cv2.filter2D(image,-1,f[:,:,i])
        
    z=z+1
  


NameError: name 'f' is not defined

In [8]:

import pandas as pd
from time import time


import tensorflow as tf
import keras
import keras.layers as layers
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.callbacks import TensorBoard


Using TensorFlow backend.


In [ ]:
model = keras.Sequential()

model.add(layers.Conv2D(filters=54, kernel_size=(3, 3), activation='relu', padding='same',input_shape=(64,64,3)))
model.add(layers.MaxPooling2D())
#model.add(layers.AveragePooling2D(input_shape=(32,32,12)))

model.add(layers.Conv2D(filters=128, kernel_size=(5, 5), activation='relu'))
model.add(layers.MaxPooling2D())

model.add(layers.Conv2D(filters=256, kernel_size=(3, 3), padding='same', activation='relu'))

model.add(layers.Conv2D(filters=256, kernel_size=(3, 3), padding='same', activation='relu'))

model.add(layers.Conv2D(filters=256, kernel_size=(3, 3), padding='same',activation='relu'))

model.add(layers.MaxPooling2D())

model.add(layers.Flatten())

model.add(layers.Dropout(0.50))

#model.add(layers.Dense(units=4096, activation='relu'))

model.add(layers.Dense(units=4096, activation='relu'))

model.add(layers.Dropout(0.50))

model.add(layers.Dense(units=4096, activation='relu'))

model.add(layers.Dense(units=class_size, activation = 'softmax'))

model.summary()

In [ ]:
model2 = keras.Sequential()

#model2.add(layers.Conv2D(filters=54, kernel_size=(3, 3), activation='relu', input_shape=(64,64,3)))
model2.add(layers.MaxPooling2D(input_shape=(64,64,54)))
#model.add(layers.AveragePooling2D(input_shape=(32,32,12)))

model2.add(layers.Conv2D(filters=128, kernel_size=(5, 5), activation='relu'))
model2.add(layers.MaxPooling2D())

model2.add(layers.Conv2D(filters=256, kernel_size=(3, 3), padding='same', activation='relu'))

#model.add(layers.Conv2D(filters=256, kernel_size=(3, 3), padding='same', activation='relu'))

#model.add(layers.Conv2D(filters=256, kernel_size=(3, 3), padding='same',activation='relu'))

model2.add(layers.MaxPooling2D())

model2.add(layers.Flatten())

model2.add(layers.Dropout(0.25))

#model.add(layers.Dense(units=4096, activation='relu'))

model2.add(layers.Dense(units=4096, activation='relu'))

model2.add(layers.Dense(units=2048, activation='relu'))

model2.add(layers.Dense(units=class_size, activation = 'softmax'))

model2.summary()

In [ ]:
model2.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(), metrics=['accuracy'])
EPOCHS = 20
BATCH_SIZE = 128

In [ ]:
#tensorboard = TensorBoard(log_dir="logs/{}".format(time()))
#model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=EPOCHS, 
#                    validation_data=validation_generator, validation_steps=validation_steps, 
#                    shuffle=True, callbacks=[tensorboard])


In [ ]:

m2 = model2.fit(figures, y,  epochs=EPOCHS);


In [ ]:
#model2.evaluate(figures2, y_test)

In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(), metrics=['accuracy'])
m= model.fit(x, y, epochs=20, validation_data = (xv,yv));

In [ ]:
#went from 2k/1k to 4k/4k and added 3rd cnn (go big or go home...)
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(), metrics=['accuracy'])
m= model.fit(x, y, epochs=20, validation_data = (xv,yv));

In [ ]:
m2= model.fit(x, y, epochs=20, validation_data = (xv,yv));

In [ ]:
plt.plot(m.history['val_acc'])
plt.plot(m.history['acc'])

In [ ]:
#added extra convolution layer

plt.plot(m.history['val_acc'])
plt.plot(m.history['acc'])

In [ ]:
#added extra convolution layer
#shank last layer from 2048 -> 1024


plt.plot(m.history['val_acc'])
plt.plot(m.history['acc'])

In [ ]:
#added extra convolution layer
#shank last layer from 2048 -> 1024
#shrank first fcc from 4096 ->2048

plt.plot(m.history['val_acc'])
plt.plot(m.history['acc'])

In [ ]:
r = 0
x2 = np.sum(x/3, axis=3, keepdims=True)


plt.figure(1)
plt.subplot(121)
plt.imshow(f[:,:,0])

plt.subplot(122)
plt.imshow(figures[r,:,:,0])
plt.figure(2)
plt.imshow(x[r])
#plt.imshow(x[r])


In [ ]:
plt.plot(m2.history['acc'])
plt.plot(m.history['acc'])

In [ ]:
hist = cv2.calcHist(img,[5],None,[256],[0,256])

In [ ]:
mask = np.zeros([64,64,3], dtype='uint8')

In [ ]:
img = xv[0:1]
color = ('b','g','r')
for i,col in enumerate(color):
    histr = cv2.calcHist(img,[0,1,2],None,[256],[0,256])
    plt.plot(histr,color = col)
    #plt.xlim([0,256])
plt.show()

In [ ]:
img.shape

In [ ]:
img = x
color = ('b','g','r')
for i,col in enumerate(color):
    histr = cv2.calcHist(img,[i],None,[256],[0,256])
    plt.plot(histr,color = col)
    #plt.xlim([0,256])
plt.show()

In [ ]:
x.shape

In [ ]:
img = xv[0]
color = ('b','g','r')
for i,col in enumerate(color):
    histr = cv2.calcHist(img,[i],None,[256],[0,256])
    plt.plot(histr,color = col)
    #plt.xlim([0,256])
plt.show()

In [ ]:
img = xv[1]
color = ('b','g','r')
for i,col in enumerate(color):
    histr = cv2.calcHist(img,[i],None,[256],[0,256])
    plt.plot(histr,color = col)
    #plt.xlim([0,256])
plt.show()

In [ ]:
img = xv[2:30]
color = ('b','g','r')
for i,col in enumerate(color):
    histr = cv2.calcHist(img,[i],None,[256],[0,256])
    plt.plot(histr,color = col)
    #plt.xlim([0,256])
    print('ya')
plt.show()

In [ ]:
img = xv[0:2]
color = ('b','g','r')
for i,col in enumerate(color):
    histr = cv2.calcHist(img,[i],None,[256],[0,256])
    plt.plot(histr,color = col)
    #plt.xlim([0,256])
plt.show()

In [ ]:
img = xv[0:1]
color = ('b','g','r')
histb = np.zeros([256,1])
histg = np.zeros([256,1])
histr = np.zeros([256,1])
for h in xv:

    histb += cv2.calcHist(h,[0],None,[256],[0,256])
    histg += cv2.calcHist(h,[1],None,[256],[0,256])
    histr += cv2.calcHist(h,[2],None,[256],[0,256])

plt.plot(histr,color = 'r')

plt.plot(histg,color = 'g')

plt.plot(histb,color = 'b')
        #plt.xlim([0,256])
plt.show()

In [ ]:
histb = np.zeros([256,1])
histg = np.zeros([256,1])
histr = np.zeros([256,1])
for h in x:

    histb += cv2.calcHist(h,[0],None,[256],[0,256])
    histg += cv2.calcHist(h,[1],None,[256],[0,256])
    histr += cv2.calcHist(h,[2],None,[256],[0,256])

plt.plot(histr,color = 'r')

plt.plot(histg,color = 'g')

plt.plot(histb,color = 'b')
        #plt.xlim([0,256])
plt.show()

In [ ]:
histr2 = histr +histr2

In [ ]:
histr2.shape

In [ ]:
plt.plot(histr2,color = col)

In [9]:
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

from keras.initializers import glorot_uniform

from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D

In [10]:
def identity_block(X, f, filters, stage, block):
    
    # Defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    # Second component of main path
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1, 1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path 
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1, 1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

In [11]:
def convolutional_block(X, f, filters, stage, block, s=2):

    # Defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value
    X_shortcut = X

    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    ##### SHORTCUT PATH #### 
    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

In [12]:
def ResNet50(input_shape = (64, 64, 3), classes = 20):
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    # Stage 3
    X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4
    X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5
    X = convolutional_block(X, f=3, filters=[512, 512, 2048], stage=5, block='a', s=2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL
    X = AveragePooling2D(pool_size=(2,2), padding='same')(X)

    # Output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [13]:
model = ResNet50(input_shape = (64, 64, 3), classes = 20)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [14]:
m= model.fit(x, y, epochs=15, batch_size = 32, validation_data = (xv,yv));

Train on 20000 samples, validate on 1000 samples
Epoch 1/15
20000/20000 [==============================] - 112s 6ms/step - loss: 6.6149e-04 - acc: 0.9984 - val_loss: 15.3122 - val_acc: 0.0500
Epoch 2/15
20000/20000 [==============================] - 99s 5ms/step - loss: 5.9605e-12 - acc: 0.9985 - val_loss: 15.3122 - val_acc: 0.0500
Epoch 3/15
20000/20000 [==============================] - 100s 5ms/step - loss: 5.9605e-12 - acc: 0.9985 - val_loss: 15.3122 - val_acc: 0.0500
Epoch 4/15
20000/20000 [==============================] - 99s 5ms/step - loss: 5.9605e-12 - acc: 0.9985 - val_loss: 15.3122 - val_acc: 0.0500
Epoch 5/15
20000/20000 [==============================] - 99s 5ms/step - loss: 5.9605e-12 - acc: 0.9985 - val_loss: 15.3122 - val_acc: 0.0500
Epoch 6/15
20000/20000 [==============================] - 99s 5ms/step - loss: 5.9605e-12 - acc: 0.9985 - val_loss: 15.3122 - val_acc: 0.0500
Epoch 7/15
20000/20000 [==============================] - 99s 5ms/step - loss: 5.9605e-12 - acc: 

KeyboardInterrupt: 

In [ ]:
model.summary()